##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Alignment of Gemma

Model alignment is the process of automatically updating a prompt based on developer-provided feedback.

This notebook shows how the [Model Alignment Python API](https://github.com/PAIR-code/model-alignment) can be used with Gemma and Gemini models to modify non-agentic prompts (e.g. ideation, extraction, summarization, etc. that don't make use of tools or external knowledge databases) based on  feedback through two different approaches described below.

Running this notebook requires a [GPU-backed Colab runtime][colab_accelerators], such as a T4, L4, or A100, which may require a paid [subscription][colab_subscriptions].

With model alignment, there are two roles that models take as part of the process, although these can be the same model in practice:

1. **Target model**: The model that the target prompt will be run with. The alignment process is meant to make the prompt work well with the target model, as determined by you.
  
  - This notebook uses the Gemma2 2B Instruction-Tuned model, loaded by Keras from weights hosted on Kaggle as the target model. Loading Gemma for this Colab requires:

    1. [Authenticating][kagglehub_auth] with a Kaggle account that has accepted the Gemma license agreement; and
    1. A [GPU-backed runtime][colab_accelerators], such as a T4, L4, or A100, which may require a paid [subscription][colab_subscriptions].

[colab_accelerators]: https://research.google.com/colaboratory/faq.html#gpu-availability
[colab_subscriptions]: https://colab.research.google.com/signup
[kagglehub_auth]: https://github.com/Kaggle/kagglehub#authenticate
[kaggle_gemma]: https://www.kaggle.com/models/keras/gemma
  
2. **Alignment model**: The model that will be used to perform the alignment steps. This model is used in the steps that perform alignment steps such as creating principles and updating the target prompt.

  - This notebook uses the Gemini Pro 1.5 model run through Google's Developer API as the alignment model. This requires you to provide an [API key](https://aistudio.google.com/app/apikey). The Gemini model is larger in size than the Gemma model, which makes it better at the alignment tasks shown in this notebook. A Gemma model larger than 2B would also suffice for the alignment of the prompts.

Alignment can be performed through two different methods, both of which are shown in this notebook:

1. Prompts are modified through guidlines (also called principles) which are curated by critiquing or rewriting of model responses.
2. Prompts are modified through direct feedback of a model response, without the intermediate step of creating principles.

The principles method has an added benefit of maintaining a list of principles by which the prompt should adhere, which can be helpful as both a resource to share and collaborate on, and as a checklist of properties that a model response can be judged agains. This does require extra alignment model processing and human involvement versus the direct feedback approach. Which approach to take is entirely up to you.

Below is a flow chart depicting the sequence of steps for both approaches.

In addition to creating principles through critiquing, principles can also be created by providing a manually-written ideal response, or by giving kudos to a well-crafted model response. For more details, see the [documentation](https://github.com/PAIR-code/model-alignment?tab=readme-ov-file#generating-principles-from-feedback-and-applying-them) and [research paper](https://arxiv.org/abs/2310.15428).

model_alignment.svg

## Notebook Setup


This notebook requires setting of three "secrets" that contain user-specific credentials for use of both models. These can be set in the "Secrets" configuration panel in this Colab notebok.
1. KAGGLE_USERNAME: Your username for kaggle.com
2. KAGGLE_KEY: A token created in your kaggle account for access to kaggle resources
3. GOOGLE_API_KEY: An API Key for using the Google Developer API for Gemini

## Setup Keras for GPU usage and JAX backend.


In [ ]:
import os
from google.colab import userdata

# Setup Keras to use jax backend and configure for successful colab GPU usage.
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

## Installation and imports

In [ ]:
# Install Keras libraries which are used by the Gemma model.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

# Install the Model Alignment library
!pip install model-alignment
from model_alignment import single_run
from model_alignment import model_helper

## Create the Gemma and Gemini model helper classses for use by Model Alignment.


In [ ]:
gemma_model = model_helper.GemmaModelHelper('gemma2_instruct_2b_en')

gemini_api_key = userdata.get('GOOGLE_API_KEY')
gemini_model = model_helper.GeminiModelHelper(gemini_api_key, 'gemini-1.5-pro')

# Simple Alignment Through Principles

This example shows aligning a simple prompt through the process of creating principles for the prompt to follow, created by providing feedback to responses generated by the model.

You first define a simple prompt to align.

You then run the prompt through the target Gemma model and see the output.

You can then provide a critique of the response in order to have the model alignment system create a generic principle that you want the model to adhere to. In this example, you ask the model alignment library to provide some possible critiques given this prompt and model response to help us brainstorm.

Once you provide the critique, a principle is created and printed out. You can repeat this process as many times as you want to update the full principles list for this model alignment task.

Once you are happy with the list of principles, you have the model alignment library update the prompt to better adhere to all of the principles you have created.

You then run the updated prompt on the target model and see the improved results.

In [ ]:
# @title Prompt to align
prompt = 'Give me a list of groceries to buy for making shrimp scampi' # @param {type:"string"}


In [ ]:
# Create the model alignment instance, using Gemma to run the prompt and Gemini to do the alignment work.
single_run_prompt = single_run.AlignableSingleRun(gemma_model, gemini_model)

In [ ]:
# Run the initial prompt with Gemma
single_run_prompt.set_model_description(prompt)
output = single_run_prompt.send_input()
print(output.text)



Here's a list of groceries you'll need for making delicious shrimp scampi:

**Seafood:**

* **Shrimp:** 1 pound large shrimp, peeled and deveined (you can use frozen if you prefer)

**Produce:**

* **Garlic:** 4-6 cloves
* **Shallots:** 1-2 shallots, finely chopped (optional, but adds a nice depth of flavor)
* **Lemon:** 1 lemon, juiced and zested
* **Parsley:** 1/4 cup chopped fresh parsley
* **Red pepper flakes:** 1/4 teaspoon (or to taste)

**Pantry:**

* **Olive oil:** 2-3 tablespoons
* **Dry white wine:** 1/2 cup (like Pinot Grigio or Sauvignon Blanc)
* **Butter:** 2 tablespoons
* **Parmesan cheese:** 1/2 cup grated (plus extra for serving)
* **Salt and black pepper:** to taste
* **Red pepper flakes:** 1/4 teaspoon (or to taste)

**Optional:**

* **Angel hair pasta:** 8 ounces (or your preferred pasta shape)
* **Bread:** for serving (optional)


Enjoy your shrimp scampi!


In [ ]:
# Generate some possible critiques of this model output using the larger model.
critiques = single_run_prompt.generate_critiques()
print(critiques)

['**Red pepper flakes** is listed twice: once under **Produce** and again under **Pantry**.', 'The quantity of shrimp is for a small portion, probably enough for 2 people. It should be specified in the instructions how many people the recipe is for.', "The instructions are not specific enough. For example, it doesn't specify what type of olive oil (extra virgin or not) or what kind of bread would pair well with the dish.", ' \n']


In [ ]:
# Make a specific critique of the prompt and response, creating a principle with the larger model.
# Then print the current principles list.
principles = single_run_prompt.critique_response('I do not want to use any alcohol in my recipe')
print(principles)

['Specify any dietary restrictions.']


In [ ]:
# Update the prompt based on the principles using the larger model and print it out.
single_run_prompt.update_model_description_from_principles()
print(single_run_prompt.get_model_description())

Give me a list of groceries to buy for making shrimp scampi. Please specify any dietary restrictions. 



In [ ]:
# Get model output from Gemma now that the prompt is updated based on the new principle.
reply = single_run_prompt.send_input()
print(reply.text)

Here's a list of groceries for making shrimp scampi, keeping in mind dietary restrictions:

**Main Ingredients:**

* **Shrimp:** 1 pound large shrimp, peeled and deveined (can be substituted with other seafood like scallops or cod)
* **Garlic:** 4-6 cloves, minced
* **Shallots:** 1-2, minced (optional, can substitute with onion)
* **Butter:** 4 tablespoons (or olive oil for a lighter option)
* **White wine:** 1/2 cup dry white wine (optional, can substitute with chicken broth or lemon juice)
* **Lemon juice:** 1 tablespoon
* **Parsley:** 1/4 cup chopped fresh parsley (for garnish)
* **Red pepper flakes:** 1/4 teaspoon (optional, for heat)

**Supporting Ingredients:**

* **Salt and black pepper:** To taste
* **Pasta:** 1 pound linguine, fettuccine, or your preferred pasta shape (gluten-free options available)
* **Parmesan cheese:** 1/2 cup grated (optional, for serving)

**Dietary Restrictions:**

* **Gluten-free:**  Choose gluten-free pasta.
* **Dairy-free:** Use dairy-free butter or o

# Alignment for Prompts with Input Variables


This example is similar to the previous example with one major difference. This example aligns a prompt that contains a **variable**. The use case for prompts with variables are when a prompt may have an end-user-specific value that needs to be plugged into it before it is sent to the model.

First, define the prompt to align with an input variable, as shown in the example below.

The word in the brackets is the variable. It represents a place where the end-user of a prompt would input their own value. In this way, this same prompt can be used to find out uses of multiple different items without writing totally separate prompts.

These types of prompts with variables can be useful for prompts that are meant to be embedded into user-facing applications where the user can set the specific value for a variable through some UI element, for example.

Notice that when you run the prompt through the target model, you provide a dictionary that sets the value of the "item" variable to the value "scissors".

You align this prompt in the same way as the previous example and you can see that as the prompt is updated by the alignment procedure, it keeps the variable in the prompt so that it can be used for any value for the "item" variable.

In [ ]:
# @title Prompt to align
prompt = 'Give three uses for the following item: {item}' # @param {type:"string"}

In [ ]:
# Create the model alignment instance, using Gemma to run the prompt and Gemini to do the alignment work.
single_run_prompt = single_run.AlignableSingleRun(gemma_model, gemini_model)

In [ ]:
# @title Set the value for the variable in the prompt
variable_value = 'scissors' # @param {type:"string"}

In [ ]:
# Run the initial prompt with Gemma with a specific value for the variable
single_run_prompt.set_model_description(prompt)
output = single_run_prompt.send_input({'item': variable_value})
print(output.text)


Here are three uses for scissors:

1. **Cutting paper:** This is the most common use for scissors, from crafting projects to schoolwork.
2. **Cutting fabric:**  Scissors are great for trimming fabric for sewing projects, crafts, or even just cutting out shapes for decorations.
3. **Cutting hair:**  While not ideal for professional use, scissors can be used to trim small amounts of hair at home, especially for trimming bangs or small sections. 


Let me know if you'd like more uses!


In [ ]:
# Make a specific critique of the prompt and response.
# Then print out the principles list from the alignment.
principles = single_run_prompt.critique_response('none of the uses are creative')
print(principles)

['Give creative uses for the item.']


In [ ]:
# Update the prompt based on the new principle.
single_run_prompt.update_model_description_from_principles()
print(single_run_prompt.get_model_description())

Give three creative uses for the following item: {item} 



In [ ]:
# Get model output now that the prompt is updated based on the new principle.
reply = single_run_prompt.send_input({'item': 'scissors'})
print(reply.text)

Here are three creative uses for scissors:

1. **Sculptural Art:**  Instead of cutting paper, use scissors to create intricate patterns and textures in clay, dough, or even foam. The precise cuts can add depth and dimension to your sculptures.

2. **Jewelry Making:**  Use scissors to cut thin strips of fabric, leather, or even wire for jewelry making. You can create delicate chains, intricate patterns, or even cut out shapes for pendants.

3. **Upcycled Crafts:**  Give old clothes or fabric scraps a new life by cutting them into shapes for patchwork quilts, decorative pillows, or even unique wall hangings.  Scissors can also be used to cut out shapes for origami or paper crafts. 


Let me know if you'd like more ideas!


# Alignment Through Direct Updates


This example shows aligning a simple prompt through direct feedback. This is a simpler approach than aligning through principle creation, which was shown above. One advantage of this approach is that it only requires one call to the alignment model as opposed to two calls for the principle-based approach. One disadvantage is that it doesn't create a standalone list of principles that captures our intent with this alignment process.

You first create a simple prompt to align.

You run the prompt through the target Gemma model and see the output.

You can then provide feedback about the response in order to have the model alignment system make a direct update to the prompt so that it better captures our intent.

Once you provide the feedback, the prompt is updated and printed out. You can repeat this process as many times as you want to continually refine the prompt.

You then run the updated prompt on the target model and see the improved results.

In [ ]:
# @title Prompt to align
prompt = 'How do I change the oil in my lawnmover?' # @param {type:"string"}

In [ ]:
# Create the model alignment instance, using Gemma to run the prompt and Gemini to do the alignment work.
single_run_prompt = single_run.AlignableSingleRun(gemma_model, gemini_model)

In [ ]:
# Run the initial prompt with Gemma
single_run_prompt.set_model_description(prompt)
output = single_run_prompt.send_input()
print(output.text)

Changing the oil in your lawnmower is a relatively simple task that can help keep your machine running smoothly. Here's a general guide, but always consult your owner's manual for specific instructions and safety precautions:

**What You'll Need:**

* **New oil:** Check your owner's manual for the correct type and amount.
* **Oil filter:**  Check your owner's manual for the correct type and size.
* **Oil filter wrench:**  This is essential for removing the old filter.
* **Drain pan:**  To catch the old oil.
* **Funnel:**  To help pour the new oil.
* **Wrench:**  To remove the drain plug.
* **Gloves:**  To protect your hands.
* **Rags or paper towels:**  For cleaning up spills.

**Steps:**

1. **Safety First:**
   * **Disconnect the spark plug:** This prevents accidental starting.
   * **Wear gloves:**  Protect your hands from hot oil.
   * **Work in a well-ventilated area:**  Oil fumes can be harmful.

2. **Locate the Oil Drain Plug:**
   * Consult your owner's manual for the exact loc

In [ ]:
# Make a specific critique of the prompt and response, causing a direct update to the prompt.
# Then print out the updated prompt
updated_prompt = single_run_prompt.update_model_description_from_feedback('make each step as simple and short as possible')
print(updated_prompt)

How do I change the oil in my lawnmover? Please make each step as short and simple as possible.


In [ ]:
# Get model output now that the prompt is updated based on the feedback.

output = single_run_prompt.send_input()
print(output.text)

## Changing Lawn Mower Oil: A Quick Guide

**Before you start:**

* **Safety first!**  Disconnect the spark plug and let the engine cool down.
* **Gather your supplies:** New oil, oil filter, wrench, drain pan, funnel, and rags.

**Steps:**

1. **Locate the oil drain plug:** Usually under the engine.
2. **Position the drain pan:** Under the plug.
3. **Remove the drain plug:** Let the oil drain completely.
4. **Replace the drain plug:** Tighten securely.
5. **Remove the oil filter:** Use a wrench.
6. **Install the new oil filter:** Hand-tighten.
7. **Add new oil:** Use the funnel to pour the correct amount (check your owner's manual).
8. **Start the engine:** Run for a few minutes, then check for leaks.
9. **Dispose of old oil properly:** Take it to a recycling center.

**Congratulations! You've changed your lawnmower oil.** 


**Important:** Always consult your owner's manual for specific instructions and oil type recommendations.
